In [12]:
try:
    import matplotlib.pyplot as plt
except ImportError:
    plt = None
    print("matplotlib não está disponível. Instale com: pip install matplotlib")

try:
    import seaborn as sns
except Exception:
    sns = None
    print("seaborn não está disponível. Instale com: pip install seaborn")

# Visualiza as primeiras linhas
print("--- Primeiras 5 linhas ---")
display(df.head())

# Verifica informações gerais (tipos e nulos)
print("\n--- Informações do DataFrame ---")
df.info()

# Verifica o balanceamento das classes
count_classes = df['Class'].value_counts()
print("\n--- Distribuição das Classes (0=Normal, 1=Fraude) ---")
print(count_classes)
print(f"\nPorcentagem de Fraudes: {count_classes[1] / len(df) * 100:.4f}%")

# Gráfico de barras simples para visualizar o desbalanceamento
if plt is not None and sns is not None:
    plt.figure(figsize=(6,4))
    sns.countplot(x='Class', data=df)
    plt.title('Distribuição de Classes: Normal (0) vs Fraude)')
    plt.show()
elif plt is not None:
    # Fallback sem seaborn: usa pandas plotting
    plt.figure(figsize=(6,4))
    df['Class'].value_counts().plot(kind='bar')
    plt.title('Distribuição de Classes: Normal (0) vs Fraude)')
    plt.show()
else:
    print("\nmatplotlib não disponível — pulando plot. Use a contagem impressa acima.")

matplotlib não está disponível. Instale com: pip install matplotlib
seaborn não está disponível. Instale com: pip install seaborn
--- Primeiras 5 linhas ---


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0



--- Informações do DataFrame ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807

In [13]:
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# --- 1. PRÉ-PROCESSAMENTO ---
# As colunas V1-V28 já estão prontas. Precisamos ajustar 'Time' e 'Amount'
# O RobustScaler é bom para dados com muitos outliers (comum em fraudes)
df_clean = df.copy()
scaler = RobustScaler()

df_clean['scaled_amount'] = scaler.fit_transform(df_clean['Amount'].values.reshape(-1, 1))
df_clean['scaled_time'] = scaler.fit_transform(df_clean['Time'].values.reshape(-1, 1))

# Removemos as originais para sobrar apenas dados numéricos escalados
df_clean.drop(['Time', 'Amount'], axis=1, inplace=True)

# --- 2. DIVISÃO EM TREINO E TESTE ---
X = df_clean.drop('Class', axis=1) # Todas as colunas menos a resposta
y = df_clean['Class']              # A resposta (0 ou 1)

# Usamos stratify=y para garantir que haja fraudes tanto no treino quanto no teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("Dados divididos!")
print(f"Treino: {X_train.shape[0]} transações")
print(f"Teste:  {X_test.shape[0]} transações")

# --- 3. TREINAMENTO DO MODELO ---
print("\nIniciando treinamento da Regressão Logística...")
model = LogisticRegression(max_iter=1000) # max_iter alto para garantir convergência
model.fit(X_train, y_train)
print("✅ Modelo treinado com sucesso!")

# --- 4. AVALIAÇÃO ---
# Vamos ver como ele se sai nos dados de teste
y_pred = model.predict(X_test)

print("\n--- RESULTADOS ---")
print(classification_report(y_test, y_pred))

# Matriz de Confusão em texto (já que o gráfico não está funcionando)
cm = confusion_matrix(y_test, y_pred)
print("\n--- MATRIZ DE CONFUSÃO (TEXTO) ---")
print(f"Transações Normais Corretas (True Negatives): {cm[0][0]}")
print(f"Falsos Alarmes (False Positives):             {cm[0][1]}")
print(f"Fraudes Perdidas (False Negatives):           {cm[1][0]}")
print(f"Fraudes Detectadas (True Positives):          {cm[1][1]}")

Dados divididos!
Treino: 227845 transações
Teste:  56962 transações

Iniciando treinamento da Regressão Logística...
✅ Modelo treinado com sucesso!

--- RESULTADOS ---
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.82      0.64      0.72        98

    accuracy                           1.00     56962
   macro avg       0.91      0.82      0.86     56962
weighted avg       1.00      1.00      1.00     56962


--- MATRIZ DE CONFUSÃO (TEXTO) ---
Transações Normais Corretas (True Negatives): 56850
Falsos Alarmes (False Positives):             14
Fraudes Perdidas (False Negatives):           35
Fraudes Detectadas (True Positives):          63


In [14]:
from sklearn.ensemble import RandomForestClassifier

# --- 1. CONFIGURAÇÃO DO RANDOM FOREST ---
# n_estimators=100: Cria 100 árvores de decisão
# n_jobs=-1: Usa todos os processadores do seu PC para ser mais rápido
# random_state=42: Garante que o resultado seja sempre o mesmo
rf_model = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42)

print("🌲 Treinando Random Forest (isso pode demorar 1 ou 2 minutos)...")
rf_model.fit(X_train, y_train)
print("✅ Treinamento concluído!")

# --- 2. AVALIAÇÃO ---
y_pred_rf = rf_model.predict(X_test)

print("\n--- RESULTADOS (RANDOM FOREST) ---")
print(classification_report(y_test, y_pred_rf))

cm_rf = confusion_matrix(y_test, y_pred_rf)
print("\n--- COMPARAÇÃO: FRAUDES ---")
print(f"Fraudes Perdidas (Antes): 35  ->  Agora: {cm_rf[1][0]}")
print(f"Fraudes Detectadas (Antes): 63 ->  Agora: {cm_rf[1][1]}")

🌲 Treinando Random Forest (isso pode demorar 1 ou 2 minutos)...
✅ Treinamento concluído!

--- RESULTADOS (RANDOM FOREST) ---
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.94      0.82      0.87        98

    accuracy                           1.00     56962
   macro avg       0.97      0.91      0.94     56962
weighted avg       1.00      1.00      1.00     56962


--- COMPARAÇÃO: FRAUDES ---
Fraudes Perdidas (Antes): 35  ->  Agora: 18
Fraudes Detectadas (Antes): 63 ->  Agora: 80


In [15]:
# --- 1. TREINAMENTO COM PESOS BALANCEADOS ---
# class_weight='balanced': O modelo dá mais "peso" para a classe minoritária (fraudes)
# max_samples=0.5: Usa menos memória para treinar mais rápido (opcional, mas ajuda)
rf_weighted = RandomForestClassifier(n_estimators=100,
                                     class_weight='balanced',
                                     n_jobs=-1,
                                     random_state=42)

print("⚖️ Treinando Random Forest com PESOS BALANCEADOS...")
rf_weighted.fit(X_train, y_train)
print("✅ Treinamento concluído!")

# --- 2. AVALIAÇÃO ---
y_pred_weighted = rf_weighted.predict(X_test)

print("\n--- RESULTADOS (COM PESOS) ---")
print(classification_report(y_test, y_pred_weighted))

cm_w = confusion_matrix(y_test, y_pred_weighted)
print("\n--- EVOLUÇÃO DAS FRAUDES PERDIDAS ---")
print(f"Regressão Logística: 35 perdidas")
print(f"Random Forest Padrão: 18 perdidas")
print(f"Random Forest Pesado: {cm_w[1][0]} perdidas")

print(f"\nFraudes Detectadas agora: {cm_w[1][1]}")

⚖️ Treinando Random Forest com PESOS BALANCEADOS...
✅ Treinamento concluído!

--- RESULTADOS (COM PESOS) ---
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.96      0.76      0.85        98

    accuracy                           1.00     56962
   macro avg       0.98      0.88      0.92     56962
weighted avg       1.00      1.00      1.00     56962


--- EVOLUÇÃO DAS FRAUDES PERDIDAS ---
Regressão Logística: 35 perdidas
Random Forest Padrão: 18 perdidas
Random Forest Pesado: 24 perdidas

Fraudes Detectadas agora: 74


In [16]:
from sklearn.metrics import accuracy_score

# 1. Pegamos as PROBABILIDADES em vez da decisão final
# (Ex: O modelo diz: "Tenho 35% de certeza que é fraude")
y_proba = rf_model.predict_proba(X_test)[:, 1]

# 2. Definimos um novo limiar de corte (agressivo)
# Se a certeza for maior que 20% (0.20), já consideramos fraude!
novo_limiar = 0.20
y_pred_ajustado = (y_proba >= novo_limiar).astype(int)

# 3. Avaliação
print(f"--- RESULTADOS COM LIMIAR DE {novo_limiar*100}% ---")
cm_ajustado = confusion_matrix(y_test, y_pred_ajustado)

print(f"Fraudes Perdidas (Padrão - 50%): 18")
print(f"Fraudes Perdidas (Novo   - 20%): {cm_ajustado[1][0]}")

print(f"\nFraudes Detectadas (Novo): {cm_ajustado[1][1]} de 98")
print(f"Falsos Alarmes (Novo):     {cm_ajustado[0][1]}")

print("\n--- Relatório Completo ---")
print(classification_report(y_test, y_pred_ajustado))

--- RESULTADOS COM LIMIAR DE 20.0% ---
Fraudes Perdidas (Padrão - 50%): 18
Fraudes Perdidas (Novo   - 20%): 12

Fraudes Detectadas (Novo): 86 de 98
Falsos Alarmes (Novo):     21

--- Relatório Completo ---
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.80      0.88      0.84        98

    accuracy                           1.00     56962
   macro avg       0.90      0.94      0.92     56962
weighted avg       1.00      1.00      1.00     56962



In [17]:
import joblib
import os

# Cria uma pasta 'models' se ela não existir
if not os.path.exists('models'):
    os.makedirs('models')

# Salva o modelo (O Random Forest que treinamos)
joblib.dump(rf_model, 'models/random_forest_v1.pkl')

# Salva o scaler (O RobustScaler que usou no início)
joblib.dump(scaler, 'models/scaler_v1.pkl')

print("💾 Arquivos salvos com sucesso na pasta 'models/':")
print("- random_forest_v1.pkl")
print("- scaler_v1.pkl")

💾 Arquivos salvos com sucesso na pasta 'models/':
- random_forest_v1.pkl
- scaler_v1.pkl


In [18]:
import joblib
import pandas as pd
import os

# --- 1. VERIFICAR O QUE O MODELO APRENDEU (Feature Importance) ---
# Extrai a importância de cada coluna
importances = rf_model.feature_importances_
feature_names = X.columns

# Cria uma tabela simples
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importancia': importances
})

# Ordena da mais importante para a menos importante
feature_importance_df = feature_importance_df.sort_values(by='Importancia', ascending=False)

print("--- TOP 10 FATORES QUE INDICAM FRAUDE ---")
# Mostra apenas as 10 principais
display(feature_importance_df.head(10))


# --- 2. SALVAR O "CÉREBRO" DA IA ---
# Cria a pasta 'models' se ela não existir
if not os.path.exists('models'):
    os.makedirs('models')

# Salva o Modelo (a lógica de decisão)
joblib.dump(rf_model, 'models/random_forest_v1.pkl')

# Salva o Scaler (a matemática de conversão de valores)
joblib.dump(scaler, 'models/scaler_v1.pkl')

print("\n✅ SUCESSO! Arquivos salvos na pasta 'models/':")
print("   - random_forest_v1.pkl")
print("   - scaler_v1.pkl")
print("\nAgora seu Notebook acabou. Vamos para a API")

--- TOP 10 FATORES QUE INDICAM FRAUDE ---


,Feature,Importancia
16,V17,0.192278
13,V14,0.139783
11,V12,0.102377
9,V10,0.071407
10,V11,0.066568
15,V16,0.058255
8,V9,0.034783
6,V7,0.033137
3,V4,0.031502
17,V18,0.023424



✅ SUCESSO! Arquivos salvos na pasta 'models/':
   - random_forest_v1.pkl
   - scaler_v1.pkl

Agora seu Notebook acabou. Podemos ir para a criação da API!
